### Saving API Token

In [5]:
%set_env QXToken=4425931e3b235a652a9a1473c168110d30705e16ba4b558db7a28f9d80374035982df54daacce62638df28262cec0f033565dea81c861c0528a894f04f152882

env: QXToken=4425931e3b235a652a9a1473c168110d30705e16ba4b558db7a28f9d80374035982df54daacce62638df28262cec0f033565dea81c861c0528a894f04f152882


### Imports

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import warnings

from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options
from qiskit_serverless import QiskitFunction, save_result, get_arguments, save_result, distribute_task, distribute_qiskit_function, IBMServerlessClient, QiskitFunction
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_transpiler_service.transpiler_service import TranspilerService
from qiskit_aer import AerSimulator

### Define Distributed task function and decorating it

In [ ]:
@distribute_task(target={"cpu": 3})                      
#The distribute_task decorator transforms a function into a distributed task
#allowing it to run on separate compute resources asynchronously and in parallel with the main context.
def transpile_parallel(circuit: QuantumCircuit, config):
    transpiled_circuit = config.run(circuit)
    return transpiled_circuit
# Get program arguments
arguments = get_arguments()
circuits = arguments.get("circuits")
backend_name = arguments.get("backend_name")
# Start process 
print("Starting timer")
start = timer()

# run distributed tasks as async function we get task references as a return type
sample_task_references = []
for circuit in range(3):
    sample_task_references.append([transpile_parallel(circuit, config) for config in configs])


# now we need to collect results from task references
results = get([task for subtasks in sample_task_references for task in subtasks])

end = timer()

# Record execution time
execution_time_serverless = end-start
print("Execution time: ", execution_time_serverless)

save_result({
    "transpiled_circuits": results,
    "execution_time" : execution_time_serverless
    
})

### Defining "circuits" to be used in the argument of the function transpile_parallel

In [7]:
# Setup 3 circuits with Efficient SU2
num_qubits = [40, 50, 60]
circuits = [EfficientSU2(nq, su2_gates=["rz","ry"], entanglement="circular", reps=1).decompose() for nq in num_qubits]

### Defining Configs to be used as a argument of transpile_parallel

In [ ]:
optimization_levels = [1 ,2, 3]
pass_managers = [{'pass_manager': generate_preset_pass_manager(optimization_level=level, backend=backend), 'optimization_level': level} for level in optimization_levels]
configs = pass_managers


### Setup Qiskit Runtime Service backend

In [8]:
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token='4425931e3b235a652a9a1473c168110d30705e16ba4b558db7a28f9d80374035982df54daacce62638df28262cec0f033565dea81c861c0528a894f04f152882',
    set_as_default=True,
    overwrite=True
)

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_brisbane")

In [ ]:
from qiskit_ibm_provider import IBMProvider

# Load your IBM Quantum account
provider = IBMProvider(token='4425931e3b235a652a9a1473c168110d30705e16ba4b558db7a28f9d80374035982df54daacce62638df28262cec0f033565dea81c861c0528a894f04f152882')
print("IBM Quantum account loaded successfully!")

### For authenticating to the remote cluster and submit the pattern for remote execution

In [ ]:
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction
serverless = IBMServerlessClient("")

### Initialization of QiskitFunction template below. The file to point is transpile_parallel.ipynb

In [21]:
transpile_parallel_function = QiskitFunction(
    title="transpile_parallel",
    entrypoint="transpile_parallel.ipynb",
    working_dir="./transpile_parallel",
    dependencies=["qiskit-transpiler-service"]
)

### Uploading on the serverless

In [ ]:
serverless.upload(transpile_parallel_function)